<a href="https://colab.research.google.com/github/SamudralaAnuhya/Q-A_langchain/blob/main/Q%26A_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install openai
# !pip install langchain-community
#!pip install langchain openai docarray
#!pip install tiktoken

#Langchain

In [ ]:
import openai
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
print(openai_api_key)

In [59]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings.openai import OpenAIEmbeddings

In [31]:
file = '/content/drive/MyDrive/Colab Notebooks/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [38]:
docs = loader.load()
docs[0]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [32]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key)
).from_loaders([loader])

In [54]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

llm = ChatOpenAI(model='gpt-4-turbo' , temperature =0.0 , openai_api_key = openai_api_key )
response = index.query(query, llm = llm)

display(Markdown(response))

Here's a table summarizing the shirts with sun protection:

| ID  | Name                                  | Fabric Composition              | Features                                                                                     |
|-----|---------------------------------------|---------------------------------|----------------------------------------------------------------------------------------------|
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | 100% Polyester                  | UPF 50+, wrinkle-resistant, front and back cape venting, two front bellows pockets           |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve  | 52% Polyester, 48% Nylon        | UPF 50+, wrinkle-free, quick-dry, front and back cape venting, two front bellows pockets     |
| 535 | Men's TropicVibe Shirt, Short-Sleeve    | 71% Nylon, 29% Polyester        | UPF 50+, wrinkle-resistant, front and back cape venting, two front bellows pockets           |
| 255 | Sun Shield Shirt                        | 78% Nylon, 22% Lycra Xtra Life fiber | UPF 50+, moisture-wicking, quick-drying, abrasion-resistant, fits over swimsuits             |

### Summary of Each Shirt:

1. **Men's Tropical Plaid Short-Sleeve Shirt (ID 618)**: This shirt offers the highest level of sun protection (UPF 50+), is made entirely of polyester, and is designed to be wrinkle-resistant. It features front and back cape venting for breathability and has two front bellows pockets.

2. **Men's Plaid Tropic Shirt, Short-Sleeve (ID 374)**: Also rated UPF 50+, this shirt blends polyester and nylon for durability and comfort. It is wrinkle-free, quickly evaporates perspiration, and includes similar venting and pocket features as the other shirts.

3. **Men's TropicVibe Shirt, Short-Sleeve (ID 535)**: This shirt combines nylon and polyester, providing UPF 50+ protection and wrinkle resistance. It includes front and back cape venting and two front bellows pockets for functionality.

4. **Sun Shield Shirt (ID 255)**: Made with a blend of nylon and Lycra Xtra Life fiber, this shirt is slightly fitted and extends to the hip. It offers UPF 50+ protection, is moisture-wicking, quick-drying, and abrasion-resistant, making it suitable for wear over swimsuits.

#step by step by using RAG (come same like above)


In [56]:
from langchain.document_loaders import CSVLoader
file = '/content/drive/MyDrive/Colab Notebooks/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()

#embeding
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# embed = embeddings.embed_query("please suggest the shirt colors available ")
# print(len(embed))
#print(embed[:3])
# o/p : [-0.014402026634070995, -0.0257216045822781, -0.009745975243918771]


#vector store
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

# query = "please suggest the shirt colors available "
# docs = db.similarity_search(query)
# len(docs)


#retrieve data
retriever = db.as_retriever()
llm = ChatOpenAI(model='gpt-4-turbo' , temperature =0.0 , openai_api_key = openai_api_key )

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)


query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

response = qa_stuff.run(query )

display(Markdown(response))




> Entering new RetrievalQA chain...

> Finished chain.


Here's a table summarizing the shirts with sun protection:

| ID  | Name                                  | Fabric Composition              | Features                                                                                     |
|-----|---------------------------------------|---------------------------------|----------------------------------------------------------------------------------------------|
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | 100% Polyester                  | UPF 50+, wrinkle-resistant, front and back cape venting, two front bellows pockets           |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | 52% Polyester, 48% Nylon        | UPF 50+, wrinkle-free, quickly evaporates perspiration, front and back cape venting, two front bellows pockets |
| 535 | Men's TropicVibe Shirt, Short-Sleeve   | 71% Nylon, 29% Polyester        | UPF 50+, wrinkle resistant, front and back cape venting, two front bellows pockets           |
| 255 | Sun Shield Shirt                       | 78% Nylon, 22% Lycra Xtra Life fiber | UPF 50+, moisture-wicking, quick-drying, abrasion resistant, fits over swimsuits             |

### Summary of Each Shirt:

1. **Men's Tropical Plaid Short-Sleeve Shirt (ID 618)**: This shirt offers the highest level of sun protection (UPF 50+), is made entirely of polyester, and features wrinkle resistance. It includes venting for cooling and practical front pockets.

2. **Men's Plaid Tropic Shirt, Short-Sleeve (ID 374)**: Designed originally for fishing, this shirt combines polyester and nylon for durability and comfort. It also features UPF 50+ sun protection, is wrinkle-free, and has efficient moisture evaporation qualities.

3. **Men's TropicVibe Shirt, Short-Sleeve (ID 535)**: This shirt is a blend of nylon and polyester, providing UPF 50+ protection. It is designed to be wrinkle-resistant and includes venting and pockets similar to the other models.

4. **Sun Shield Shirt (ID 255)**: This shirt is unique with its high nylon and Lycra blend, making it suitable for wear over swimsuits. It offers UPF 50+ protection, is quick-drying, moisture-wicking, and abrasion-resistant, making it ideal for active use.

#both langchain and retriever for comparison

In [ ]:
#langchain
file = '/content/drive/MyDrive/Colab Notebooks/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings(openai_api_key=openai_api_key)
).from_loaders([loader])

query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

llm = ChatOpenAI(model='gpt-4-turbo' , temperature =0.0 , openai_api_key = openai_api_key )
response = index.query(query, llm = llm)

display(Markdown(response))



#retrival (embeding , vectorstore , retrieve)
file = '/content/drive/MyDrive/Colab Notebooks/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

#vector store
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

#retrieve data
retriever = db.as_retriever()
llm = ChatOpenAI(model='gpt-4-turbo' , temperature =0.0 , openai_api_key = openai_api_key )

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)


query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

response = qa_stuff.run(query )

display(Markdown(response))
